In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [24]:
crowdfunding_2018 = pd.read_csv('ks-projects-201801.csv')
crowdfunding_2016 = pd.read_csv('ks-projects-201612.csv', encoding='ISO-8859-1')

crowdfunding_2016.drop(['Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16'], axis=1, inplace=True)

crowdfunding_2016.head()

crowdfunding_2016['ID']

C:\Users\sivak\AppData\Local\Temp\ipykernel_26792\2937080395.py:2: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  crowdfunding_2016 = pd.read_csv('ks-projects-201612.csv', encoding='ISO-8859-1')


KeyError: 'ID'

In [23]:
merged_df = pd.merge(crowdfunding_2016, crowdfunding_2018, on='ID', how='inner')

KeyError: 'ID'